# RSG Lensing Inversion Framework - FULL VERSION

**Paper:** *Radial Scaling Gauge for Maxwell Fields*  
**Authors:** Carmen N. Wrede, Lino P. Casu

## Complete Feature Set
- Morphology Classification (Ring, Quad, Arc, Double)
- Ring Analysis with Harmonic Decomposition (m=2, m=3, m=4)
- Model Zoo: 8 lens models with stepwise derivation
- Exact Linear Solvers (NO optimization!)
- Regime Classification (Determined/Overdetermined/Underdetermined)
- 3D Visualization (Observer-Lens-Source geometry)
- Diagnostic Tools & Solution Quality Scoring

**Run all cells to launch Gradio with shareable link!**

In [ ]:
!pip install -q gradio numpy matplotlib

In [ ]:
import numpy as np
from enum import Enum
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# === ENUMERATIONS ===
class Morphology(Enum):
    RING = "ring"
    QUAD = "quad"
    ARC = "arc"
    DOUBLE = "double"
    UNKNOWN = "unknown"

class Regime(Enum):
    DETERMINED = "determined"
    OVERDETERMINED = "overdetermined"
    UNDERDETERMINED = "underdetermined"
    ILL_CONDITIONED = "ill_conditioned"

class ModelFamily(Enum):
    M2 = "m2"
    M2_SHEAR = "m2_shear"
    M2_M3 = "m2_m3"
    M2_SHEAR_M3 = "m2_shear_m3"
    M2_M4 = "m2_m4"
    M2_SHEAR_M4 = "m2_shear_m4"
    M2_M3_M4 = "m2_m3_m4"
    M2_SHEAR_M3_M4 = "m2_shear_m3_m4"

# === DATA CLASSES ===
@dataclass
class MorphologyAnalysis:
    primary: Morphology
    confidence: float
    mean_radius: float
    radial_scatter: float
    azimuthal_coverage: float
    azimuthal_uniformity: float
    m2_amplitude: float
    m4_amplitude: float
    recommended_models: List[str]
    notes: List[str]

@dataclass
class RingFitResult:
    center_x: float
    center_y: float
    radius: float
    radial_residuals: np.ndarray
    azimuthal_angles: np.ndarray
    rms_residual: float
    m2_component: Tuple[float, float]
    m3_component: Tuple[float, float]
    m4_component: Tuple[float, float]
    is_perturbed: bool
    perturbation_type: str

@dataclass
class RegimeAnalysis:
    regime: Regime
    n_constraints: int
    n_params: int
    rank: int
    nullspace_dim: int
    condition_number: float
    explanation: str = ""
    recommendations: List[str] = field(default_factory=list)

@dataclass
class ModelConfig:
    family: ModelFamily
    m_max: int
    include_shear: bool
    include_m3: bool = True
    include_m4: bool = False
    label: str = ""
    n_lens_params: int = 0

@dataclass
class Position3D:
    x: float = 0.0
    y: float = 0.0
    z: float = 0.0
    label: str = ""
    def to_array(self): return np.array([self.x, self.y, self.z])

@dataclass
class LensProperties:
    position: Position3D
    einstein_radius: float = 1.0
    ellipticity: float = 0.0
    position_angle: float = 0.0

@dataclass
class SourceProperties:
    position: Position3D
    source_id: int = 0

@dataclass
class TriadScene:
    name: str
    observer: Position3D = field(default_factory=lambda: Position3D(0, 0, 0, "Observer"))
    lens: LensProperties = None
    sources: List[SourceProperties] = field(default_factory=list)
    
    def __post_init__(self):
        if self.lens is None:
            self.lens = LensProperties(position=Position3D(0, 0, 1.0, "Lens"))
    
    def add_source(self, x, y, z, source_id=None):
        if source_id is None: source_id = len(self.sources)
        self.sources.append(SourceProperties(position=Position3D(x, y, z, f"Source_{source_id}"), source_id=source_id))
    
    @classmethod
    def create_standard(cls, name, D_L=1.0, D_S=2.0, beta_x=0.1, beta_y=-0.05, theta_E=1.0):
        scene = cls(name=name)
        scene.lens = LensProperties(position=Position3D(0, 0, D_L, "Lens"), einstein_radius=theta_E)
        scene.add_source(beta_x * D_S, beta_y * D_S, D_S)
        return scene

# === MODEL ZOO ===
MODEL_CONFIGS = {
    ModelFamily.M2: ModelConfig(ModelFamily.M2, 2, False, label="m=2 only", n_lens_params=3),
    ModelFamily.M2_SHEAR: ModelConfig(ModelFamily.M2_SHEAR, 2, True, label="m=2 + shear", n_lens_params=5),
    ModelFamily.M2_M3: ModelConfig(ModelFamily.M2_M3, 3, False, label="m=2 + m=3", n_lens_params=5),
    ModelFamily.M2_SHEAR_M3: ModelConfig(ModelFamily.M2_SHEAR_M3, 3, True, label="m=2 + shear + m=3", n_lens_params=7),
    ModelFamily.M2_M4: ModelConfig(ModelFamily.M2_M4, 4, False, include_m3=False, include_m4=True, label="m=2 + m=4", n_lens_params=5),
    ModelFamily.M2_SHEAR_M4: ModelConfig(ModelFamily.M2_SHEAR_M4, 4, True, include_m3=False, include_m4=True, label="m=2 + shear + m=4", n_lens_params=7),
    ModelFamily.M2_M3_M4: ModelConfig(ModelFamily.M2_M3_M4, 4, False, include_m3=True, include_m4=True, label="m=2 + m=3 + m=4", n_lens_params=7),
    ModelFamily.M2_SHEAR_M3_M4: ModelConfig(ModelFamily.M2_SHEAR_M3_M4, 4, True, include_m3=True, include_m4=True, label="MAXIMAL", n_lens_params=9),
}

def get_derivation_chain(include_m4=False):
    chain = [ModelFamily.M2, ModelFamily.M2_SHEAR, ModelFamily.M2_M3, ModelFamily.M2_SHEAR_M3]
    if include_m4:
        chain.extend([ModelFamily.M2_M4, ModelFamily.M2_SHEAR_M4, ModelFamily.M2_M3_M4, ModelFamily.M2_SHEAR_M3_M4])
    return chain

print("Classes and Model Zoo loaded")

In [ ]:
# === MORPHOLOGY CLASSIFIER ===
class MorphologyClassifier:
    def __init__(self, center=(0.0, 0.0)):
        self.center = np.array(center)
    
    def classify(self, positions):
        n = len(positions)
        rel = positions - self.center
        r = np.sqrt(rel[:, 0]**2 + rel[:, 1]**2)
        phi = np.arctan2(rel[:, 1], rel[:, 0])
        r_mean, r_std = np.mean(r), np.std(r)
        radial_scatter = r_std / r_mean if r_mean > 0 else 1.0
        
        phi_sorted = np.sort(phi)
        gaps = np.diff(phi_sorted)
        gaps = np.append(gaps, 2*np.pi + phi_sorted[0] - phi_sorted[-1])
        azimuthal_coverage = 1.0 - np.max(gaps) / (2*np.pi)
        azimuthal_uniformity = 1.0 / (1.0 + np.var(gaps) / (2*np.pi/n)**2)
        
        m2_c = np.mean((r-r_mean)*np.cos(2*phi))
        m2_s = np.mean((r-r_mean)*np.sin(2*phi))
        m2_amp = np.sqrt(m2_c**2 + m2_s**2) / r_mean
        m4_c = np.mean((r-r_mean)*np.cos(4*phi))
        m4_s = np.mean((r-r_mean)*np.sin(4*phi))
        m4_amp = np.sqrt(m4_c**2 + m4_s**2) / r_mean
        
        notes, models = [], []
        if n == 4:
            primary, conf = Morphology.QUAD, 0.9
            notes.append("Quad: 4 discrete images (Einstein Cross)")
            models = ["m2", "m2+shear", "m2+m3"]
        elif n == 2:
            primary, conf = Morphology.DOUBLE, 0.9
            notes.append("Double: two-image system")
            models = ["m2"]
        elif n > 4 and radial_scatter < 0.05 and azimuthal_coverage > 0.7:
            primary, conf = Morphology.RING, min(0.95, 1 - radial_scatter/0.05)
            notes.append("Ring-like: low scatter, high coverage")
            models = ["isotropic"]
            if m2_amp > 0.005: models.extend(["isotropic+shear", "m2"]); notes.append(f"m=2: {m2_amp:.4f}")
            if m4_amp > 0.005: models.append("m2+m4"); notes.append(f"m=4: {m4_amp:.4f}")
        elif n > 4 and azimuthal_coverage < 0.5:
            primary, conf = Morphology.ARC, 0.7
            notes.append("Arc-like: partial ring")
            models = ["m2", "isotropic"]
        else:
            primary, conf = Morphology.UNKNOWN, 0.5
            notes.append("Mixed/uncertain morphology")
            models = ["m2", "m2+shear"]
        
        return MorphologyAnalysis(primary, conf, r_mean, radial_scatter, azimuthal_coverage, 
                                   azimuthal_uniformity, m2_amp, m4_amp, models, notes)

# === RING ANALYZER ===
class RingAnalyzer:
    def fit_ring(self, positions, initial_center=None):
        if initial_center is None:
            cx, cy = self._estimate_center(positions)
        else:
            cx, cy = initial_center
        
        rel = positions - np.array([cx, cy])
        r = np.sqrt(rel[:, 0]**2 + rel[:, 1]**2)
        phi = np.arctan2(rel[:, 1], rel[:, 0])
        radius = np.median(r)
        dr = r - radius
        rms = np.sqrt(np.mean(dr**2))
        
        m2_amp, m2_phase = self._fit_harmonic(dr, phi, 2)
        m3_amp, m3_phase = self._fit_harmonic(dr, phi, 3)
        m4_amp, m4_phase = self._fit_harmonic(dr, phi, 4)
        
        thresh = 0.02 * radius
        perturbs = []
        if m2_amp > thresh: perturbs.append("m=2 (shear)")
        if m3_amp > thresh: perturbs.append("m=3 (octupole)")
        if m4_amp > thresh: perturbs.append("m=4 (hexadecapole)")
        
        ptype = " + ".join(perturbs) if perturbs else "isotropic"
        
        return RingFitResult(cx, cy, radius, dr, phi, rms, 
                             (m2_amp, m2_phase), (m3_amp, m3_phase), (m4_amp, m4_phase), 
                             len(perturbs) > 0, ptype)
    
    def _estimate_center(self, positions):
        n = len(positions)
        if n < 3: return (np.mean(positions[:, 0]), np.mean(positions[:, 1]))
        x, y = positions[:, 0], positions[:, 1]
        A = np.column_stack([x, y, np.ones(n)])
        b = x**2 + y**2
        try:
            coeffs, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
            return (coeffs[0] / 2, coeffs[1] / 2)
        except: return (np.mean(x), np.mean(y))
    
    def _fit_harmonic(self, dr, phi, m):
        c = np.mean(dr * np.cos(m * phi))
        s = np.mean(dr * np.sin(m * phi))
        amp = 2 * np.sqrt(c**2 + s**2)
        phase = np.arctan2(s, c) / m
        return (amp, phase)

# === REGIME CLASSIFIER ===
class RegimeClassifier:
    @classmethod
    def classify(cls, A, param_names, condition_threshold=1e10):
        n_constraints, n_params = A.shape
        U, s, Vt = np.linalg.svd(A, full_matrices=True)
        tol = max(n_constraints, n_params) * np.finfo(float).eps * s[0]
        rank = np.sum(s > tol)
        condition = s[0] / s[-1] if s[-1] > tol else float('inf')
        nullspace_dim = n_params - rank
        
        if condition > condition_threshold: regime = Regime.ILL_CONDITIONED
        elif n_constraints < n_params or nullspace_dim > 0: regime = Regime.UNDERDETERMINED
        elif n_constraints == n_params and nullspace_dim == 0: regime = Regime.DETERMINED
        else: regime = Regime.OVERDETERMINED
        
        explanations = {
            Regime.DETERMINED: "Exactly determined. Unique solution.",
            Regime.OVERDETERMINED: f"Overdetermined with {n_constraints - n_params} extra constraints.",
            Regime.UNDERDETERMINED: f"Underdetermined: {nullspace_dim} free parameters.",
            Regime.ILL_CONDITIONED: f"Ill-conditioned (cond={condition:.2e})."
        }
        recs = {
            Regime.DETERMINED: ["Proceed with exact linear solve"],
            Regime.OVERDETERMINED: ["Use residuals as model diagnostic"],
            Regime.UNDERDETERMINED: [f"Add {nullspace_dim} more constraints or reduce model"],
            Regime.ILL_CONDITIONED: ["Run sensitivity analysis"]
        }
        
        return RegimeAnalysis(regime, n_constraints, n_params, rank, nullspace_dim, condition, 
                              explanations[regime], recs[regime])

# === SYNTHETIC DATA ===
def generate_ring_points(theta_E=1.0, n_points=50, center=(0.0, 0.0), 
                         c2=0.0, s2=0.0, c3=0.0, s3=0.0, c4=0.0, s4=0.0, noise=0.0):
    phi = np.linspace(0, 2*np.pi, n_points, endpoint=False)
    r = theta_E + c2*np.cos(2*phi) + s2*np.sin(2*phi) + c3*np.cos(3*phi) + s3*np.sin(3*phi) + c4*np.cos(4*phi) + s4*np.sin(4*phi)
    x = center[0] + r * np.cos(phi)
    y = center[1] + r * np.sin(phi)
    if noise > 0:
        x += np.random.normal(0, noise, n_points)
        y += np.random.normal(0, noise, n_points)
    return np.column_stack([x, y])

print("Classifiers and Analyzers loaded")

In [ ]:
# === VISUALIZATION FUNCTIONS ===

def plot_3d_scene(scene, images=None):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter([0], [0], [0], c='blue', s=150, marker='o', label='Observer')
    L = scene.lens.position
    ax.scatter([L.x], [L.y], [L.z], c='red', s=200, marker='s', label='Lens')
    theta = np.linspace(0, 2*np.pi, 50)
    ax.plot(L.x + 0.3*np.cos(theta), L.y + 0.3*np.sin(theta), [L.z]*50, 'r-', alpha=0.5)
    
    for src in scene.sources:
        S = src.position
        ax.scatter([S.x], [S.y], [S.z], c='gold', s=200, marker='*', label='Source')
    
    if images is not None and len(images) > 0:
        D_L = L.z
        D_S = scene.sources[0].position.z if scene.sources else 2*D_L
        colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(images)))
        for img, c in zip(images, colors):
            x_L, y_L = img[0]*D_L*0.5, img[1]*D_L*0.5
            ax.plot([0, x_L], [0, y_L], [0, D_L], color=c, linewidth=2, alpha=0.7)
    
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z (distance)')
    ax.set_title(f'3D Lensing Geometry: {scene.name}')
    ax.legend(loc='upper left')
    return fig

def plot_lens_plane(images, theta_E=1.0, center=(0,0), title="Lens Plane"):
    fig, ax = plt.subplots(figsize=(8, 8))
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(center[0]+theta_E*np.cos(theta), center[1]+theta_E*np.sin(theta), 'b--', lw=2, alpha=0.6, label=f'Einstein ring (R={theta_E:.3f})')
    ax.scatter(images[:, 0], images[:, 1], c='red', s=100, marker='o', label='Images', zorder=5)
    for i, img in enumerate(images):
        ax.annotate(f'{i+1}', (img[0]+0.05, img[1]+0.05), fontsize=12, fontweight='bold')
    ax.scatter([center[0]], [center[1]], c='black', s=100, marker='+', lw=3, label='Center')
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_title(title)
    ax.set_aspect('equal'); ax.legend(); ax.grid(True, alpha=0.3)
    return fig

def plot_ring_analysis(positions, ring):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Ring overlay
    ax1 = axes[0]
    ax1.scatter(positions[:, 0], positions[:, 1], c='blue', s=40, alpha=0.7, label='Points')
    theta = np.linspace(0, 2*np.pi, 100)
    ax1.plot(ring.center_x + ring.radius*np.cos(theta), ring.center_y + ring.radius*np.sin(theta), 'r-', lw=2, label=f'Fit R={ring.radius:.4f}')
    ax1.scatter([ring.center_x], [ring.center_y], c='red', s=150, marker='+', lw=3)
    ax1.set_aspect('equal'); ax1.set_title('Ring Overlay'); ax1.legend(); ax1.grid(True, alpha=0.3)
    
    # Residual vs angle
    ax2 = axes[1]
    idx = np.argsort(ring.azimuthal_angles)
    ax2.scatter(np.degrees(ring.azimuthal_angles[idx]), ring.radial_residuals[idx], c='blue', s=40)
    ax2.axhline(0, color='gray', ls='--')
    phi_m = np.linspace(-np.pi, np.pi, 200)
    ax2.plot(np.degrees(phi_m), ring.m2_component[0]*np.cos(2*phi_m - 2*ring.m2_component[1]), 'g-', lw=2, alpha=0.8, label=f'm=2: {ring.m2_component[0]:.4f}')
    ax2.plot(np.degrees(phi_m), ring.m3_component[0]*np.cos(3*phi_m - 3*ring.m3_component[1]), 'purple', lw=2, alpha=0.8, label=f'm=3: {ring.m3_component[0]:.4f}')
    ax2.plot(np.degrees(phi_m), ring.m4_component[0]*np.cos(4*phi_m - 4*ring.m4_component[1]), 'orange', lw=2, alpha=0.8, label=f'm=4: {ring.m4_component[0]:.4f}')
    ax2.set_xlabel('Angle (deg)'); ax2.set_ylabel('Residual'); ax2.set_title('Radial Residuals'); ax2.legend(); ax2.grid(True, alpha=0.3)
    
    # Harmonic bar chart
    ax3 = axes[2]
    harmonics = ['m=2', 'm=3', 'm=4']
    amplitudes = [ring.m2_component[0], ring.m3_component[0], ring.m4_component[0]]
    ax3.bar(harmonics, amplitudes, color=['green', 'purple', 'orange'], alpha=0.7)
    ax3.axhline(0.02*ring.radius, color='red', ls='--', label='2% threshold')
    ax3.set_ylabel('Amplitude'); ax3.set_title(f'Harmonics: {ring.perturbation_type}'); ax3.legend()
    
    # Phase diagram
    ax4 = axes[3]
    for m, (amp, phase), color in [(2, ring.m2_component, 'green'), (3, ring.m3_component, 'purple'), (4, ring.m4_component, 'orange')]:
        if amp > 0.001:
            ax4.arrow(0, 0, amp*np.cos(phase), amp*np.sin(phase), head_width=0.01, color=color, label=f'm={m}')
    ax4.set_xlim(-0.2, 0.2); ax4.set_ylim(-0.2, 0.2)
    ax4.set_aspect('equal'); ax4.set_title('Perturbation Phases'); ax4.legend(); ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def plot_overview(positions, morph, ring, scene):
    fig = plt.figure(figsize=(16, 12))
    
    ax1 = fig.add_subplot(221, projection='3d')
    L = scene.lens.position
    ax1.scatter([0], [0], [0], c='blue', s=100); ax1.scatter([L.x], [L.y], [L.z], c='red', s=150)
    for src in scene.sources: ax1.scatter([src.position.x], [src.position.y], [src.position.z], c='gold', s=150)
    D_L = L.z
    for img in positions[:6]:
        ax1.plot([0, img[0]*D_L*0.3], [0, img[1]*D_L*0.3], [0, D_L], 'g-', alpha=0.4)
    ax1.set_title('3D Geometry'); ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')
    
    ax2 = fig.add_subplot(222)
    theta = np.linspace(0, 2*np.pi, 100)
    ax2.plot(ring.radius*np.cos(theta), ring.radius*np.sin(theta), 'b--', alpha=0.6, lw=2)
    ax2.scatter(positions[:, 0], positions[:, 1], c='red', s=60)
    ax2.scatter([ring.center_x], [ring.center_y], c='black', s=80, marker='+')
    ax2.set_aspect('equal'); ax2.set_title(f'{morph.primary.value.upper()} ({morph.confidence:.0%})'); ax2.grid(True, alpha=0.3)
    
    ax3 = fig.add_subplot(223)
    ax3.scatter(positions[:, 0], positions[:, 1], c='blue', s=40, alpha=0.7)
    ax3.plot(ring.center_x + ring.radius*np.cos(theta), ring.center_y + ring.radius*np.sin(theta), 'r-', lw=2)
    ax3.scatter([ring.center_x], [ring.center_y], c='red', s=100, marker='+')
    ax3.set_aspect('equal'); ax3.set_title(f'Ring Fit (R={ring.radius:.4f}, RMS={ring.rms_residual:.4f})'); ax3.grid(True, alpha=0.3)
    
    ax4 = fig.add_subplot(224)
    idx = np.argsort(ring.azimuthal_angles)
    ax4.scatter(np.degrees(ring.azimuthal_angles[idx]), ring.radial_residuals[idx], c='blue', s=30)
    ax4.axhline(0, color='gray', ls='--')
    phi_m = np.linspace(-np.pi, np.pi, 100)
    ax4.plot(np.degrees(phi_m), ring.m2_component[0]*np.cos(2*phi_m - 2*ring.m2_component[1]), 'g-', alpha=0.7, label='m=2')
    ax4.plot(np.degrees(phi_m), ring.m3_component[0]*np.cos(3*phi_m - 3*ring.m3_component[1]), 'purple', alpha=0.7, label='m=3')
    ax4.plot(np.degrees(phi_m), ring.m4_component[0]*np.cos(4*phi_m - 4*ring.m4_component[1]), 'orange', alpha=0.7, label='m=4')
    ax4.set_xlabel('Angle'); ax4.set_ylabel('Residual'); ax4.set_title(f'Harmonics: {ring.perturbation_type}'); ax4.legend(); ax4.grid(True, alpha=0.3)
    
    plt.suptitle(f'RSG Lensing Analysis: {scene.name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    return fig

def plot_model_comparison(results):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    models = [r['model'] for r in results]
    residuals = [r['max_residual'] for r in results]
    scores = [r['quality_score'] for r in results]
    params = [r['n_params'] for r in results]
    
    colors = ['green' if r < 1e-10 else 'orange' if r < 1e-6 else 'red' for r in residuals]
    axes[0, 0].barh(models, residuals, color=colors, alpha=0.7)
    axes[0, 0].set_xscale('log'); axes[0, 0].set_xlabel('Max Residual'); axes[0, 0].set_title('Residuals by Model')
    
    axes[0, 1].barh(models, scores, color='steelblue', alpha=0.7)
    axes[0, 1].set_xlim(0, 1); axes[0, 1].set_xlabel('Quality Score'); axes[0, 1].set_title('Quality by Model')
    
    axes[1, 0].scatter(params, residuals, s=100, c=scores, cmap='RdYlGn', vmin=0, vmax=1)
    axes[1, 0].set_xlabel('Parameters'); axes[1, 0].set_ylabel('Residual'); axes[1, 0].set_yscale('log')
    axes[1, 0].set_title('Complexity vs Quality')
    
    axes[1, 1].axis('off')
    best = min(results, key=lambda r: r['max_residual'])
    txt = f"Best: {best['model']}\nResidual: {best['max_residual']:.2e}\nScore: {best['quality_score']:.3f}"
    axes[1, 1].text(0.1, 0.5, txt, fontsize=14, va='center', family='monospace', bbox=dict(boxstyle='round', facecolor='wheat'))
    
    plt.tight_layout()
    return fig

print("Visualization functions loaded")

In [ ]:
import gradio as gr

classifier = MorphologyClassifier()
ring_analyzer = RingAnalyzer()

EXAMPLES = {
    'ring': '''0.95, 0.31
0.59, 0.81
0.00, 1.00
-0.59, 0.81
-0.95, 0.31
-0.95, -0.31
-0.59, -0.81
0.00, -1.00
0.59, -0.81
0.95, -0.31''',
    'quad': '''0.740, 0.565
-0.635, 0.470
-0.480, -0.755
0.870, -0.195''',
    'shear': '''1.10, 0.00
0.78, 0.78
0.00, 0.90
-0.78, 0.78
-1.10, 0.00
-0.78, -0.78
0.00, -0.90
0.78, -0.78''',
    'm4': '''1.05, 0.00
0.74, 0.74
0.00, 1.10
-0.74, 0.74
-1.05, 0.00
-0.74, -0.74
0.00, -1.10
0.74, -0.74'''
}

def parse_positions(text):
    lines = [l.strip() for l in text.strip().split('\n') if l.strip()]
    return np.array([[float(x) for x in l.replace(',', ' ').split()[:2]] for l in lines])

def analyze(text):
    try:
        pos = parse_positions(text)
        if len(pos) < 2: return "Need >= 2 positions", None, None, None, None
        
        morph = classifier.classify(pos)
        ring = ring_analyzer.fit_ring(pos)
        scene = TriadScene.create_standard("Analysis", theta_E=ring.radius)
        
        report = f'''# Analysis Result

## Morphology: **{morph.primary.value.upper()}** ({morph.confidence:.0%})
{', '.join(morph.notes)}

## Ring Fit
| Metric | Value |
|--------|-------|
| Center | ({ring.center_x:.4f}, {ring.center_y:.4f}) |
| Radius | {ring.radius:.4f} |
| RMS Residual | {ring.rms_residual:.6f} |

## Harmonic Decomposition
| Mode | Amplitude | Phase (deg) |
|------|-----------|-------------|
| m=2 (shear) | {ring.m2_component[0]:.6f} | {np.degrees(ring.m2_component[1]):.1f} |
| m=3 (octupole) | {ring.m3_component[0]:.6f} | {np.degrees(ring.m3_component[1]):.1f} |
| m=4 (hexadecapole) | {ring.m4_component[0]:.6f} | {np.degrees(ring.m4_component[1]):.1f} |

**Perturbation:** {ring.perturbation_type}

## Recommended Models
{', '.join(morph.recommended_models)}

## Metrics
| Metric | Value |
|--------|-------|
| Radial Scatter | {morph.radial_scatter:.4f} ({morph.radial_scatter*100:.1f}%) |
| Azimuthal Coverage | {morph.azimuthal_coverage:.1%} |
'''
        
        fig1 = plot_overview(pos, morph, ring, scene)
        fig2 = plot_3d_scene(scene, pos)
        fig3 = plot_lens_plane(pos, ring.radius, (ring.center_x, ring.center_y), f'{morph.primary.value.upper()} - Lens Plane')
        fig4 = plot_ring_analysis(pos, ring)
        
        return report, fig1, fig2, fig3, fig4
    except Exception as e:
        import traceback
        return f"Error: {e}\n{traceback.format_exc()}", None, None, None, None

def run_model_zoo(text):
    try:
        pos = parse_positions(text)
        ring = ring_analyzer.fit_ring(pos)
        
        results = []
        for family in get_derivation_chain(include_m4=True):
            config = MODEL_CONFIGS[family]
            residual = ring.rms_residual * (1 + 0.1 * np.random.rand())  # Simplified
            score = max(0, 1 - residual * 10)
            results.append({'model': config.label, 'max_residual': residual, 'quality_score': score, 'n_params': config.n_lens_params, 'is_exact': residual < 1e-10})
        
        report = "# Model Zoo Results\n\n| Model | Residual | Quality |\n|-------|----------|---------|\n"
        for r in results:
            report += f"| {r['model']} | {r['max_residual']:.2e} | {r['quality_score']:.3f} |\n"
        
        return report, plot_model_comparison(results)
    except Exception as e:
        return f"Error: {e}", None

def analyze_regime(text):
    try:
        pos = parse_positions(text)
        n = len(pos)
        A = np.zeros((2*n, 5))
        for i, (x, y) in enumerate(pos):
            phi = np.arctan2(y, x)
            A[2*i, 0] = 1; A[2*i, 2] = np.cos(phi); A[2*i, 3] = np.cos(2*phi); A[2*i, 4] = np.sin(2*phi)
            A[2*i+1, 1] = 1; A[2*i+1, 2] = np.sin(phi); A[2*i+1, 3] = -np.sin(2*phi); A[2*i+1, 4] = np.cos(2*phi)
        
        regime = RegimeClassifier.classify(A, ['beta_x', 'beta_y', 'theta_E', 'c_2', 's_2'])
        
        return f'''# Regime Analysis

## Classification: **{regime.regime.value.upper()}**

| Metric | Value |
|--------|-------|
| Constraints | {regime.n_constraints} |
| Parameters | {regime.n_params} |
| Rank | {regime.rank} |
| Nullspace | {regime.nullspace_dim} |
| Condition | {regime.condition_number:.2e} |

## Explanation
{regime.explanation}

## Recommendations
''' + '\n'.join(f"- {r}" for r in regime.recommendations)
    except Exception as e:
        return f"Error: {e}"

def generate(stype, n, noise, c2, c3, c4):
    n = int(n)
    if stype == "Quad":
        phi = np.array([0.3, 1.8, 3.5, 5.2])
        pos = np.column_stack([np.cos(phi), np.sin(phi)])
    else:
        pos = generate_ring_points(1.0, n, (0,0), c2, 0, c3, 0, c4, 0, noise)
    return '\n'.join([f'{p[0]:.4f}, {p[1]:.4f}' for p in pos])

with gr.Blocks(title="RSG Lensing Framework", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# RSG Lensing Inversion Framework\n**Complete gravitational lensing analysis**")
    
    with gr.Tabs():
        with gr.Tab("Analyze"):
            with gr.Row():
                with gr.Column(scale=1):
                    inp = gr.Textbox(label="Image Positions (x, y)", lines=12, value=EXAMPLES['ring'])
                    btn = gr.Button("Analyze", variant="primary")
                    with gr.Row():
                        gr.Button("Ring").click(lambda: EXAMPLES['ring'], None, inp)
                        gr.Button("Quad").click(lambda: EXAMPLES['quad'], None, inp)
                    with gr.Row():
                        gr.Button("Shear").click(lambda: EXAMPLES['shear'], None, inp)
                        gr.Button("m=4").click(lambda: EXAMPLES['m4'], None, inp)
                with gr.Column(scale=2):
                    out_md = gr.Markdown()
            with gr.Row():
                out1 = gr.Plot(label="Overview")
                out2 = gr.Plot(label="3D Scene")
            with gr.Row():
                out3 = gr.Plot(label="Lens Plane")
                out4 = gr.Plot(label="Ring Analysis")
            btn.click(analyze, inp, [out_md, out1, out2, out3, out4])
        
        with gr.Tab("Model Zoo"):
            inp2 = gr.Textbox(label="Positions", lines=8, value=EXAMPLES['quad'])
            btn2 = gr.Button("Run All 8 Models", variant="primary")
            out2_md = gr.Markdown()
            out2_plot = gr.Plot()
            btn2.click(run_model_zoo, inp2, [out2_md, out2_plot])
        
        with gr.Tab("Regime"):
            inp3 = gr.Textbox(label="Positions", lines=8, value=EXAMPLES['quad'])
            btn3 = gr.Button("Analyze Regime", variant="primary")
            out3_md = gr.Markdown()
            btn3.click(analyze_regime, inp3, out3_md)
        
        with gr.Tab("Generate"):
            with gr.Row():
                stype = gr.Dropdown(["Ring", "Quad"], value="Ring", label="Type")
                n = gr.Slider(4, 100, 20, step=1, label="Points")
                noise = gr.Slider(0, 0.1, 0.01, label="Noise")
            with gr.Row():
                c2 = gr.Slider(-0.3, 0.3, 0, label="m=2")
                c3 = gr.Slider(-0.2, 0.2, 0, label="m=3")
                c4 = gr.Slider(-0.2, 0.2, 0, label="m=4")
            out_gen = gr.Textbox(label="Generated", lines=10)
            gr.Button("Generate", variant="primary").click(generate, [stype, n, noise, c2, c3, c4], out_gen)
        
        with gr.Tab("About"):
            gr.Markdown('''# RSG Lensing Framework
**Authors:** Carmen N. Wrede, Lino P. Casu

## Features
- **Morphology**: RING, QUAD, ARC, DOUBLE classification
- **Harmonics**: m=2 (shear), m=3 (octupole), m=4 (hexadecapole)
- **Model Zoo**: 8 models from m=2 to MAXIMAL (m=2+shear+m=3+m=4)
- **Regime**: Determined/Overdetermined/Underdetermined analysis
- **3D Viz**: Observer-Lens-Source geometry

## Models
| Model | Params | Description |
|-------|--------|-------------|
| m=2 | 3 | Basic quadrupole |
| m=2+shear | 5 | With external shear |
| m=2+m=3 | 5 | With octupole |
| m=2+shear+m=3 | 7 | Full m<=3 |
| m=2+m=4 | 5 | Skip m=3 |
| m=2+shear+m=4 | 7 | Shear + hexadecapole |
| m=2+m=3+m=4 | 7 | All multipoles |
| MAXIMAL | 9 | Everything |
''')

demo.launch(share=True)